## CMF 1
Institut für Musikinformatik und Musikwissenschaft – Wintersemester 2025–26
### Woche 09 – Übungen

### Aufgabe 00.

Überlege dir eine Frage zum Inhalt der Vorlesung, z. B. über einen Punkt, der unklar geblieben ist oder über etwas, worüber du gerne mehr wissen möchtest.   

### Aufgabe 01. Error-Detection mithilfe der DTW-Distanz / des DTW-Scores.

In der Vorlesung haben wir die Librosa-DTW-Implementierung genutzt, um Chromagramme – und über diese Chromagramme Partituren und Aufnahmen – zu alignen.\
Das Ziel dieser Aufgabe ist es, DTW zu nutzen, um Fehler in Partituren zu erkennen.

Eine Referenz-Datei *chopin_score_reference.musicxml* enthält die Partitur des Préludes in e-Moll von Chopin.\
Unter den Dateien *chopin_score1.musicxml*, *chopin_score2.musicxml* und *chopin_score3.musicxml* sollen – mithilfe des DTW-Verfahrens – die Dateien gefunden werden, die einen Fehler enthalten.

In [1]:
import librosa
import music21 as m21
import numpy as np
import matplotlib.pyplot as plt

1) Ein paar nützliche Funktionen aus der Vorlesung:

In [2]:
def symbolic_chromagram(score, frames_per_quarter_note):

    chordified_score = score.chordify()
    # Akkord-Liste des zu analysierenden Scores
    score_chords = list(chordified_score.flatten().getElementsByClass(m21.chord.Chord))

    # Dauer des zu analysierenden Scores (in Viertelnoten ausgedrückt)
    total_duration = int(score_chords[-1].offset + score_chords[-1].quarterLength)

    # Anzahl der Analyse-Frames im Chromagramm
    number_frames_in_chromagram = total_duration*frames_per_quarter_note

    # Initialisierung des Chromagramms als 2-dimensionales Array, das nur Nullen enthält
    chromagram = np.zeros((12, number_frames_in_chromagram))

    # Iterierung über alle Akkorde in der Akkord-Liste
    for chord in score_chords:

        # Start-Frame im Chromagramm für den aktuellen Akkord
        current_chord_offset = chord.offset
        start_in_chromagram = int(current_chord_offset*frames_per_quarter_note)

        # End-Frame im Chromagramm für den aktuellen Akkord
        chord_duration = chord.quarterLength
        end_in_chromagram = int(start_in_chromagram + chord_duration*frames_per_quarter_note)

        # Bestimmung aller im aktuellen Akkord enthaltenen Pitch-Classes
        pitch_classes_current_chord = []
        for note in chord:
            pitch_class = note.pitch.pitchClass
            pitch_classes_current_chord.append(pitch_class)

        # Eintrag des aktuellen Akkords in das Chromagramm
        for time_index in np.arange(start_in_chromagram, end_in_chromagram):

            for pitch_class in pitch_classes_current_chord:
                chromagram[pitch_class, time_index] = chromagram[pitch_class, time_index] + 1
    
    return chromagram

In [3]:
def cos_distance(vector1, vector2): 

    cos_dist = 1 - (((np.dot(vector1, vector2) / (np.linalg.norm(vector1)*np.linalg.norm(vector2))) + 1)/2)

    return cos_dist

In [4]:
def cost_matrix(chromagram1, chromagram2):

    num_frames_chromagram1 = chromagram1.shape[1]
    num_frames_chromagram2 = chromagram2.shape[1]

    # Initialisierung der Cost-Matrix 
    CM = np.zeros([num_frames_chromagram1, num_frames_chromagram2])

    for i in range(num_frames_chromagram1):
        for j in range(num_frames_chromagram2):
            CM[i, j] = cos_distance(chromagram1[:, i], chromagram2[:, j])

    # Falls die Berechnung einer Cost-Matrix NaN (Not a Number) ergeben sollte, setzen wir den Wert auf 0
    CM[np.isnan(CM)] = 1

    return CM

2) Erstellung der Chromagramme aller Scores:

In [8]:
frames_per_quarter_note = 24

score_reference = m21.converter.parse('chopin_score_reference.musicxml')
chromagram_score_reference = symbolic_chromagram(score=score_reference, frames_per_quarter_note=frames_per_quarter_note)

score1 = m21.converter.parse('chopin_score1.musicxml')
chromagram_score1 = symbolic_chromagram(score=score1, frames_per_quarter_note=frames_per_quarter_note)

score2 = m21.converter.parse('chopin_score2.musicxml')
chromagram_score2 = symbolic_chromagram(score=score2, frames_per_quarter_note=frames_per_quarter_note)

score3 = m21.converter.parse('chopin_score3.musicxml')
chromagram_score3 = symbolic_chromagram(score=score3, frames_per_quarter_note=frames_per_quarter_note)

3) Schlage eine Methode vor, wie man DTW dazu nutzen kann, um zu bestimmen, welche Dateien Fehler enthalten, das heißt von der Referenz-Datei abweichende Stellen haben.

In [29]:
def findError(reference, score):
    lenReference = reference.shape[1]
    lenScore = score.shape[1]
    CM = cost_matrix(reference, score)
    for i in range(lenReference):
        if CM[i,i] != 0:
            print(f"found error at {i}")

In [43]:
findError(chromagram_score_reference,chromagram_score3)

found error at 576
found error at 577
found error at 578
found error at 579
found error at 580
found error at 581
found error at 582
found error at 583
found error at 584
found error at 585
found error at 586
found error at 587
found error at 588
found error at 589
found error at 590
found error at 591
found error at 592
found error at 593
found error at 594
found error at 595
found error at 596
found error at 597
found error at 598
found error at 599


### Aufgabe 02. Error-Detection mithilfe der Levenshtein-Distanz.

Das Ziel dieser Aufgabe ist dasselbe wie das der vorherigen Aufgabe. Als Werkzeug soll diesmal die Levenshtein genutzt werden.

In [33]:
import librosa
import music21 as m21
import numpy as np
import matplotlib.pyplot as plt

1) Unsere Levenshtein-Distanz-Funktion aus der Vorlesung:

In [41]:
def levenshtein_distance(sequence1, sequence2):
    M = len(sequence1) + 1
    N = len(sequence2) + 1

    # Matrix zum Abspeichern der Distanzen zwischen allen möglichen Sequenz-Anfängen
    D = np.zeros([M, N])

    for i in range(M):
        for j in range(N):

            if i == 0:
                D[i, j] = j

            elif j == 0:
                D[i, j] = i

            elif sequence1[i-1] == sequence2[j-1]:
                D[i, j] = D[i-1, j-1]

            else:
                min_value = min(D[i-1, j-1], D[i-1, j], D[i, j-1])
                D[i, j] = min_value + 1 

    return D[i, j]

2) Erstellung der verschiedenen Chromagramme:

In [35]:
frames_per_quarter_note = 24

score_reference = m21.converter.parse('chopin_score_reference.musicxml')
chromagram_score_reference = symbolic_chromagram(score=score_reference, frames_per_quarter_note=frames_per_quarter_note)

score1 = m21.converter.parse('chopin_score1.musicxml')
chromagram_score1 = symbolic_chromagram(score=score1, frames_per_quarter_note=frames_per_quarter_note)

score2 = m21.converter.parse('chopin_score2.musicxml')
chromagram_score2 = symbolic_chromagram(score=score2, frames_per_quarter_note=frames_per_quarter_note)

score3 = m21.converter.parse('chopin_score3.musicxml')
chromagram_score3 = symbolic_chromagram(score=score3, frames_per_quarter_note=frames_per_quarter_note)

3) Überlege dir eine Variante, bzw. eine Anwendung der Levenshtein-Distanz, um zu bestimmen, welche der Dateien Fehler enthalten. Was sagt der gefundene Score aus?

In [39]:
print(chromagram_score_reference.shape)

(12, 1176)


In [79]:
chromagram_score_reference[0]

array([0., 0., 0., ..., 0., 0., 0.], shape=(1176,))

In [74]:
levenshtein_distance(chromagram_score_reference[:,34],chromagram_score3[:,0])

np.float64(3.0)

In [77]:
def levensteinError(reference, score):
    lenReference = reference.shape[1]
    lenScore = score.shape[1]
    for i in range(lenReference):
        for j in range(lenScore):
            if levenshtein_distance(reference[:,i], score[:,j]) != 0:
                print(f"found error at {i}")